In [1]:
REQUEST_ID = 1011
AOI = 'POLYGON ((-94.74437712304278 42.10688312211514, -94.78351591698579 42.10713783113902, -94.78351591698579 42.13795006926095, -94.74437712304278 42.13756819109344, -94.74437712304278 42.10688312211514))'
START_DATE = '2020-07-07'
END_DATE = '2020-07-14'

In [2]:
# !pip3 install -q tslearn

In [3]:
import os
import shutil
import json
import time
import cv2
import rasterio
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio.mask
import shapely

from tqdm.notebook import tqdm
from os.path import join, basename, split
from scipy.ndimage import rotate
from rasterio import features
from rasterio.merge import merge
from rasterio.warp import (aligned_target,
                           calculate_default_transform,
                           reproject,
                           Resampling)
from shapely.geometry import Polygon, shape, LinearRing, box
import shapely.wkt
from pathlib import Path
from datetime import datetime
import matplotlib.pylab as plt
import seaborn as sns

#from tslearn.clustering import TimeSeriesKMeans
#from tslearn.utils import to_time_series_dataset
import rasterio.mask as riomask

from code.download.utils import get_tiles, _check_folder, check_nodata, get_min_clouds, transform_resolution
from code.download.load_tiles import load_images
from code.index_research import combine_bands, calculate_ndvi, calculate_tcari, calculate_msr_g
from code.plant_stress import PlantStress

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
default_crs = 'EPSG:4326'

polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

In [5]:
SEED = 66
NB_USER = os.getenv('NB_USER')
BASE = f"/home/{NB_USER}/work"

API_KEY = os.path.join(BASE, ".secret/sentinel2_google_api_key.json")
sentinel_tiles_path = os.path.join(BASE, "notebooks/pbdnn/sentinel2grid.geojson")
LOAD_DIR = os.path.join(BASE, "satellite_imagery")
RESULTS_DIR = os.path.join(BASE, "results/pbdnn")
PBD_DIR = os.path.join(BASE, "notebooks/pbdnn")

BANDS = {'B04','B08', 'TCI'}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0, 'CLOUDY_PIXEL_PERCENTAGE': 10.0, }
PRODUCT_TYPE = 'L2A'

for pbd_file in os.listdir(os.path.join(BASE, 'results/pbdnn')):
    if pbd_file.startswith(str(REQUEST_ID)+'_') or pbd_file.startswith('demo_'+str(REQUEST_ID)+'_'):
        PB_PATH = os.path.join(BASE, 'results/pbdnn', pbd_file)
        break
        
    else:
        # should be trigger PBDNN notebook excecution? PBDNN pipeline?
        PB_PATH = aoi_filename
        
PB_PATH

'1626684458.40807_aoi.geojson'

In [6]:
PB_PATH = '/home/jovyan/work/notebooks/dd/iowa_fields.geojson'

In [7]:
b04_tiles, b08_tiles, tci_tiles = [], [], []
date_tile_info = get_tiles(aoi_filename, sentinel_tiles_path)

In [8]:
loadings = load_images(API_KEY, date_tile_info.tileID.values, START_DATE, END_DATE, LOAD_DIR, BANDS, CONSTRAINTS, PRODUCT_TYPE)
checked = check_nodata(loadings, PRODUCT_TYPE)

try:
    checked = get_min_clouds(checked)
except Exception as e:
    print(f'No clean raster found for period from {start} to {end}, skipping, {e}')

for i, tile in date_tile_info.iterrows():
    try:
        tile_folder = Path(checked[tile.tileID])
        print(f'filtered: {tile_folder}')
    except Exception as ex:
        print(ex)

b04_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B04_10m.jp2' in filename][0]
b08_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B08_10m.jp2' in filename][0]
tci_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'TCI_10m.jp2' in filename][0]

filtered: /home/jovyan/work/satellite_imagery/S2A_MSIL2A_20200713T170901_N0214_R112_T15TUG_20200713T213310


In [9]:
b04_tile

'/home/jovyan/work/satellite_imagery/S2A_MSIL2A_20200713T170901_N0214_R112_T15TUG_20200713T213310/T15TUG_20200713T170901_B04_10m.jp2'

In [10]:
with rasterio.open(tci_tile) as src:
    tci_image = src.read()

In [11]:
os.makedirs(f'{BASE}/results/dd/plant_stress/{REQUEST_ID}', exist_ok=True)
ndvi_path = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/{REQUEST_ID}_ndvi.tif'
if not os.path.exists(ndvi_path):
    calculate_ndvi(b04_tile, b08_tile, out_path=ndvi_path, nodata=np.nan)

In [12]:
def transform_crs(data_path, save_path, dst_crs='EPSG:4326', resolution=(10, 10)):
    with rasterio.open(data_path) as src:
        if resolution is None:
            transform, width, height = calculate_default_transform(src.crs,
                                                                   dst_crs,
                                                                   src.width,
                                                                   src.height,
                                                                   *src.bounds)
        else:
            transform, width, height = calculate_default_transform(src.crs,
                                                                   dst_crs,
                                                                   src.width,
                                                                   src.height,
                                                                   *src.bounds,
                                                                   resolution=resolution)
        kwargs = src.meta.copy()
        kwargs.update({'crs': dst_crs,
                       'transform': transform,
                       'width': width,
                       'height': height})
        with rasterio.open(save_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
        dst.close()
    src.close()

    return save_path

In [13]:
def stitch_tiles(paths, out_raster_path='test.tif'):
    tiles = []
    tmp_files = []
    
    for i, path in enumerate(paths):
        if i == 0:
            file = rasterio.open(path)
            meta, crs = file.meta, file.crs
        else:
            tmp_path = path.replace(
                '.jp2', '_tmp.jp2').replace('.tif', '_tmp.tif')
            crs_transformed = transform_crs(path, tmp_path, 
                                            dst_crs=crs, 
                                            resolution=None)
            tmp_files.append(crs_transformed)
            file = rasterio.open(crs_transformed)
        tiles.append(file)
            
    tile_arr, transform = merge(tiles, method='last')
    
    
    meta.update({"driver": "GTiff",
                 "height": tile_arr.shape[1],
                 "width": tile_arr.shape[2],
                 "transform": transform,
                 "crs": crs})
    
    if '.jp2' in out_raster_path:
        out_raster_path = out_raster_path.replace('.jp2', '_merged.tif')
    else:
        out_raster_path = out_raster_path.replace('.tif', '_merged.tif')
    print(f'saved raster {out_raster_path}')

    for tile in tiles:
        tile.close()
        
    for tmp_file in tmp_files:
        try:
            os.remove(tmp_file)
        except FileNotFoundError:
            print(f'Tile {tmp_file} was removed or renamed, skipping')
        
    with rasterio.open(out_raster_path, "w", **meta) as dst:
        dst.write(tile_arr)
    
    return out_raster_path

In [14]:
fields = gpd.read_file(PB_PATH)

In [15]:
min_ndvi = 0.3
z_score = 5
z_score_anom = 1
rasters_to_stitch = []
p_bar = tqdm(fields.iterrows(), total=len(fields))
colors = {"Normal Growth": (0, 0, 0), "Anomaly": (182, 10, 28)}
ps = PlantStress(min_ndvi=min_ndvi, noise_z_score=z_score, anomaly_z_score=z_score_anom)

for i, record in enumerate(p_bar):
    idx, row = record
    field = fields.to_crs(src.crs).iloc[idx].geometry
    out_raster = ps.segment_field(field, ndvi_path, START_DATE, END_DATE, REQUEST_ID, idx)
    rasters_to_stitch.append(out_raster)

In [16]:
rasters_to_stitch

['/home/jovyan/work/results/dd/plant_stress/1011/1011_field_1011_0.tif',
 '/home/jovyan/work/results/dd/plant_stress/1011/1011_field_1011_1.tif',
 '/home/jovyan/work/results/dd/plant_stress/1011/1011_field_1011_2.tif',
 '/home/jovyan/work/results/dd/plant_stress/1011/1011_field_1011_3.tif',
 '/home/jovyan/work/results/dd/plant_stress/1011/1011_field_1011_4.tif',
 '/home/jovyan/work/results/dd/plant_stress/1011/1011_field_1011_5.tif',
 '/home/jovyan/work/results/dd/plant_stress/1011/1011_field_1011_6.tif',
 '/home/jovyan/work/results/dd/plant_stress/1011/1011_field_1011_7.tif',
 '/home/jovyan/work/results/dd/plant_stress/1011/1011_field_1011_8.tif']

In [17]:
path = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/{REQUEST_ID}_merged.tif'
stitch_tiles(rasters_to_stitch, path)

saved raster /home/jovyan/work/results/dd/plant_stress/1011/1011_merged_merged.tif


'/home/jovyan/work/results/dd/plant_stress/1011/1011_merged_merged.tif'

In [18]:
remove_aux_rasters = True

if remove_aux_rasters:
    
    try:
        shutil.rmtree(tile_folder)
    except FileNotFoundError:
        print(f'Error removing directory {tile_folder}')
        
    try:
        os.remove(ndvi_path)
    except FileNotFoundError:
        print(f'Error removing file {ndvi_path}')
        
    for i in range(len(b04_tiles)):
        ndvi_p = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/ndvi_{i}.tif'
        try:
            os.remove(ndvi_p)
        except FileNotFoundError:
            print(f'Error removing file {ndvi_p}')
            
    for raster_path in rasters_to_stitch:
        try:
            os.remove(raster_path)
        except FileNotFoundError:
            print(f'Error removing file {raster_path}')